<a href = "https://www.pieriantraining.com"><img src="../PT Centered Purple.png"> </a>

<em style="text-align:center">Copyrighted by Pierian Training</em>

# Additional Chains
There are many specialized additional chains you can exlore in the documentation! Let's look at a more commonly used on, whic allows us to directly perform Document QA with just a few simple calls.

### Connect to Document Vector Store

In [5]:
import os
from dotenv import load_dotenv
load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")

In [6]:
from langchain_openai import OpenAIEmbeddings
#from langchain.vectorstores import Chroma
from langchain_chroma import Chroma

In [7]:
embedding_function = OpenAIEmbeddings()

In [8]:
db_connection = Chroma(
    persist_directory='../01-Data-Connections/US_Constitution/',
    embedding_function=embedding_function
)

### Load QA Models

In [12]:
from langchain.chains.question_answering import load_qa_chain
# The QA chain with sources is like the QA chain but
# it returns some metadata related to the source documents
# where the answer was found.
from langchain.chains.qa_with_sources import load_qa_with_sources_chain

In [10]:
from langchain_openai import ChatOpenAI

In [11]:
llm = ChatOpenAI(temperature=0)

In [13]:
# chain_type: stuff = we will enter the context in to a prompt; that the usual type
chain = load_qa_chain(llm,chain_type='stuff')

In [14]:
question = "What is the 15th amendment?"

In [16]:
# We first need to perform the similarity search and get the documents
docs = db_connection.similarity_search(question)

In [17]:
docs # [Document(), ...]

[Document(metadata={'source': 'some_data/US_Constitution.txt'}, page_content='Section 3\nNo person shall be a Senator or Representative in Congress, or elector of President and Vice-President, or hold any office, civil or military, under the United States, or under any State, who, having previously taken an oath, as a member of Congress, or as an officer of the United States, or as a member of any State legislature, or as an executive or judicial officer of any State, to support the Constitution of the United States, shall have engaged in insurrection or rebellion against the same, or given aid or comfort to the enemies thereof. But Congress may by a vote of two-thirds of each House, remove such disability.\n\nSection 4\nThe validity of the public debt of the United States, authorized by law, including debts incurred for payment of pensions and bounties for services in suppressing insurrection or rebellion, shall not be questioned. But neither the United States nor any State shall assu

In [18]:
# We insert the documents found in the similarity search and the question into the chain
chain.invoke({"input_documents":docs,"question":question})["output_text"] # The 15th Amendment to the United States Constitution prohibits the denial or abridgment...

'The 15th Amendment to the United States Constitution prohibits the denial or abridgment of the right to vote based on race, color, or previous condition of servitude.'

### Get Original Source Context Information

In [19]:
# The QA chain with sources is like the QA chain but
# it returns some metadata related to the source documents
# where the answer was found.
chain = load_qa_with_sources_chain(llm,chain_type='stuff')

In [20]:
query = "What is the 14th amendment?"

In [21]:
docs = db_connection.similarity_search(query)

In [22]:
chain.invoke({"input_documents":docs,"question":query})["output_text"] # The 14th Amendment states that all persons...

'The 14th Amendment states that all persons born or naturalized in the United States are citizens of the United States and that no State shall make or enforce any law which shall abridge the privileges or immunities of citizens of the United States, nor deprive any person of life, liberty, or property without due process of law, nor deny to any person within its jurisdiction the equal protection of the laws.\nSOURCES: some_data/US_Constitution.txt'